In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [43]:
import pandas as pd
import dask.dataframe as dd
from transformers import AutoTokenizer
import pyarrow as pa

In [5]:
#X_train = pd.read_csv('drive/MyDrive/BERT Sentiment/CSVs/X_train_full.csv')
X_test = pd.read_csv('drive/MyDrive/BERT Sentiment/CSVs/X_test_full.csv')

In [6]:
#X_train.columns = ['index','review']
X_test.columns = ['index','review']

In [7]:
#X_train.set_index(keys='index',inplace=True)
X_test.set_index('index',inplace=True)
X_test.head()

,review
index,
886662,Great resource: This is a great down to earth ...
112830,What an awesome dentist!\n\nThe staff is very ...
2320316,good while it lasted (for my baby): This was a...
566879,I love this place. I didn't eat here for awhil...
1666132,doesn't play on a DVD player?: I'm confused. T...


In [8]:
# Load HuggingFace Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [9]:
# Tokenize reviews
#train_inputs = tokenizer(list(X_train['review']),max_length=True,padding=True)

In [10]:
test_inputs = tokenizer(list(X_test['review']),truncation=True,padding=True)

In [11]:
test_inputs.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
# Examining reviews with unknown tokens
has_unknown = []
for i, input in enumerate(test_inputs['input_ids']):
  for tok in input:
    # Searches for an [UNK] token
    if tok == 100:
      # Prints entire line with padding characters removed
      print(f"input {i}: {tokenizer.decode(input).replace(' [PAD]','')}.")
      has_unknown.append(i)
      break

input 34945: [CLS] fat? copy? i think not... : do these look fat to you? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 1do these look like bratz? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! she shows some skin? yes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! is that horribly wrong? no!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ( do you show skin? yes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! is this doll perfect? [UNK]!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! [SEP].
input 138420: [CLS] [UNK] [SEP].
input 263693: [CLS] 3. 75 star review ( i'm an engineer..... what can i say, i like precision! ) : - ) \ n \ nback for another run at vapiano's, and another solid effort. the trick is to go early before the banking crowd overruns the place..... so 1130 is ideal. \ n \ ni had the americana pizza, with marinara, fresh mozzerella, pepperoni and italian sausage, and i had them add onions and mushrooms. neopolitan style, not \ " ny \ " style ( with that cru

In [48]:
# Checking the originals of those reviews
for index in has_unknown:
  print(X_test.iloc[index]['review'])

FAT? COPY? I THINK NOT...: Do these look fat to you?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1DO THESE LOOK LIKE BRATZ?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!SHE SHOWS SOME SKIN?YES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!IS THAT HORRIBLY WRONG?NO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!(do you show skin?YES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!IS THIS DOLL PERFECT?YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm1
3.75 Star Review (I'm an Engineer.....what can I say, I like precision!)  :-)\n\nBack for another run at Vapiano's, and another solid effort. The trick is to go early before the Banking crowd overruns the place.....so 1130 i

In [49]:
untokenizable_sample = 'mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm'
len(untokenizable_sample)


157

It looks like the primary source of unknown tokens are excessively repeated characters, such as in "mmmmmmmmmm..." with 157 m's.

In [19]:
type(test_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [37]:
test_input_ddf = dd.from_dict({'input_ids': test_inputs['input_ids'],
                               'attention_mask': test_inputs['attention_mask'],
                               'token_type_ids': test_inputs['token_type_ids']},
                              npartitions=10)

In [40]:
test_input_ddf.head(5)


,input_ids,attention_mask,token_type_ids
0,"[101, 2307, 7692, 1024, 2023, 2003, 1037, 2307...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[101, 2054, 2019, 12476, 24385, 999, 1032, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[101, 2204, 2096, 2009, 6354, 1006, 2005, 2026...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[101, 1045, 2293, 2023, 2173, 1012, 1045, 2134...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[101, 2987, 1005, 1056, 2377, 2006, 1037, 4966...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [44]:
schema = pa.schema([('input_ids', pa.list_(pa.int64())),
                    ('attention_mask', pa.list_(pa.int64())),
                    ('token_type_ids', pa.list_(pa.int64()))])

In [45]:
test_input_ddf.to_parquet('drive/MyDrive/BERT Sentiment/CSVs/test_inputs.parquet',schema=schema)
